# Data Mining
## Exercise 3 -  A-priori algorithm and association rules 

**Student: David Sánchez Marín**


###  Mining Frequent Itemsets - A-priori algorithm for k up to 2

In [1]:
import pyspark
import os
import math
import random
import sys

In [2]:
# Preliminary start-up code:
spark_home = os.environ.get('SPARK_HOME', None)
print ( spark_home )
sc = pyspark.SparkContext('local[*]')
print ( sc )

None
<SparkContext master=local[*] appName=pyspark-shell>


In [3]:
# From a transaction in a single string with items separated by white spaces
# generate a list with the items of the transaction
def parseTransaction( trans ):
    return trans.strip().split('\t')

# Compute the rdd with frequent singletonsets (L_1)
def computeL1 ( rddtrans, numtrans, theta ):
  rddtemp = rddtrans.flatMap( lambda trans : [ (it,1) for it in trans ] ).reduceByKey( lambda a,b : a+b  )
  rddtemp2 = rddtemp.filter( lambda x: x[0] != '')

  rddL1 = rddtemp2.filter( lambda x : (float(x[1])/numtrans) >= theta )
  return rddL1

# This is a function to map transactions to transactions with only items in L1,
# something that can increase the efficiency when computing L2
def filterOutL1( transseq, L1 ):
    for trans in transseq:
       yield [ it for it in trans if (it in L1) ]


In [4]:
def generateC2( seqoftransactions,  L1  ):
    
    for trans1 in seqoftransactions:     
        itemsetlist = []
        if len(trans1) > 1:
            trans2 = trans1.copy()
            for it1 in trans1:
                trans2.remove(it1)
                for it2 in trans2:
                    pair = (it1, it2)
                    itemsetlist.append(pair)
        yield itemsetlist


In [5]:
# And finally, the function for computing L_2 from L_1 and the rdd with
# the set of transactions
# This function should use your previous generateC2 function to compute L2 from C2
#
def computeL2( rddtrans, numtrans, L1, theta, verbose ):
    #   Map the transactions in rddtrans to the set of candidate frquenquent pairs (C2) 
    #   using the previous function generateC2      
        
    candidatePairsrdd = rddtrans.mapPartitions( lambda trans: generateC2(trans, L1onlyitems) )
    if verbose:
        print ("\n  candidatePairsrdd: ", candidatePairsrdd.take(10))

    candidatePairsfilteredrdd = candidatePairsrdd.filter( lambda x: len(x) > 0 ).map( lambda x: x[0])
    if verbose:
        print ("\n  candidatePairsfilteredrdd: ", candidatePairsfilteredrdd.take(10))

    #  Count number of occurences in the transactions for  each pair in C2. 
    #  Use flatpMap and reduceByKey to get the final rdd
    #rddL2temp = candidatePairsfilteredrdd.flatMap( lambda trans : [ (it,1) for it in trans ] ).reduceByKey( lambda a,b : a+b  ) 
    rddL2temp = candidatePairsfilteredrdd.map( lambda it : (it, 1) ).reduceByKey( lambda a,b : a+b  ) 
    if verbose:
        print ("\n  rddL2temp: ", rddL2temp.take(10))
    
    # Finally, filter out from the previous rdd those pairs with frequency below theta
    rddL2 = rddL2temp.filter( lambda x : (float(x[1])/numtrans) >= theta ) 
    
    return rddL2

In [6]:
# Load information in Dataset

rddlines = sc.textFile( "../data/1979.csv").map(parseTransaction)
numtrans = rddlines.count()
print ("Number of registers: ", numtrans)
print(rddlines.take(5))

Number of registers:  430941
[['0', '19790101', '197901', '1979', '1979.0027', '', '', '', '', '', '', '', '', '', '', 'AFR', 'AFRICA', 'AFR', '', '', '', '', '', '', '', '1', '040', '040', '04', '1', '1', '9', '1', '9', '5.52631578947368', '', '', '', '', '0', '0', '0', '', '', '', '', '0', '0', '0', '', '', '', '', '0', '0', '', '20130203'], ['1', '19790101', '197901', '1979', '1979.0027', '', '', '', '', '', '', '', '', '', '', 'AGR', 'FARMER', '', '', '', '', '', 'AGR', '', '', '1', '030', '030', '03', '1', '4', '10', '1', '10', '10.9792284866469', '', '', '', '', '0', '0', '0', '1', 'Nigeria', 'NI', 'NI', '10', '8', '0', '1', 'Nigeria', 'NI', 'NI', '10', '8', '0', '20130203'], ['2', '19790101', '197901', '1979', '1979.0027', '', '', '', '', '', '', '', '', '', '', 'AGR', 'FARMER', '', '', '', '', '', 'AGR', '', '', '1', '100', '100', '10', '3', '-5', '10', '1', '10', '10.9792284866469', '', '', '', '', '0', '0', '0', '1', 'Nigeria', 'NI', 'NI', '10', '8', '0', '1', 'Nigeria', 'NI'

In [7]:
# Select fields
rddtrans = rddlines.map(lambda x: [x[15],x[27],x[44]])
rddtrans.take(10)

[['AFR', '040', ''],
 ['AGR', '030', 'NI'],
 ['AGR', '100', 'NI'],
 ['CHN', '043', 'CH'],
 ['COP', '190', 'US'],
 ['CVL', '046', 'IS'],
 ['EGYEDU', '040', ''],
 ['GOV', '020', 'TU'],
 ['GOV', '040', 'CH'],
 ['GOV', '040', 'TW']]

In [10]:
# Generate L1 and L2 with differents theta values

thetaList = [0.0100, 0.0075, 0.0050]
for theta in thetaList:
    print ("************************************************************************************************************")
    print ("                                           THETA = ", theta)
    print ("************************************************************************************************************\n")
    # First, let's compute the frequent singleton sets
    #       
    rddL1 = computeL1( rddtrans, float(numtrans), theta  )
    L1 = rddL1.collect()
    print ("L1 singleton with frequency information: \n", L1 )

    # Compute a version without the frequency information:
    #
    L1onlyitems = rddL1.map( lambda it : it[0] ).collect()
    print ( "\nL1 only items: \n", L1onlyitems )

    if theta == 0.010:
        # Show details of calculation on first loop
        verbose = 1
    else:
        verbose = 0
    
    # Next, compute frequent pairs (L2) from frequent items (L1)
    # We  need to work with the collected back version (to the driver) of the L1 set
    # (This will be Ok as far as L1 is small enough to fit in the memory
    # of every single machine ). We use mapPartitions to distribute only one function call per partition,
    # avoiding the overhead of executing many functions with the parameter L1
    #
    L1filteredtransrdd = rddtrans.mapPartitions( lambda transseq : filterOutL1( transseq, L1onlyitems )  )

    # Let's check if the filtered transactions are correct:
    #
    print ("\nTransactions with only L1 items : \n", L1filteredtransrdd.take(10) )


    # Next, compute the frequent pairs (L_2)
    #
    rddL2 = computeL2( L1filteredtransrdd, float(numtrans), L1onlyitems, theta, verbose )
    print ( "\nL2 Pairs with frequency information : ", rddL2.collect() )

    L2onlyitems = rddL2.map( lambda it : it[0] ).collect()
    print ( "\nL2 only items: ", L2onlyitems )
    

************************************************************************************************************
                                           THETA =  0.01
************************************************************************************************************

L1 singleton with frequency information: 
 [('CH', 25151), ('190', 17254), ('051', 21278), ('LEG', 4850), ('112', 7218), ('USA', 19484), ('036', 17954), ('111', 8545), ('110', 4808), ('040', 30785), ('US', 32488), ('CVL', 4575), ('042', 50824), ('ZI', 5869), ('057', 10552), ('RS', 20365), ('SF', 5247), ('VNM', 12557), ('120', 8467), ('FR', 6313), ('013', 6065), ('GBR', 6890), ('043', 48844), ('046', 25938), ('IS', 14897), ('GOV', 29969), ('020', 17660), ('IR', 16888), ('CB', 4996), ('MIL', 7756), ('VM', 17120), ('173', 8070), ('010', 25975), ('030', 7116), ('CHN', 15506), ('PL', 5223), ('IRN', 8552), ('JA', 5558), ('KHM', 4727), ('EG', 9533), ('RUS', 11368), ('080', 4396), ('050', 5186), ('EGY', 6679), ('ISR', 8928